In [3]:
head(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [4]:
dim(iris)

[1] 150   5

In [5]:
str(iris)

'data.frame':	150 obs. of  5 variables:
 $ Sepal.Length: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ Sepal.Width : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ Petal.Length: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ Petal.Width : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ Species     : Factor w/ 3 levels "setosa","versicolor",..: 1 1 1 1 1 1 1 1 1 1 ...


In [6]:
# Splitting the dataset into the Training set and Test set
# install.packages('caTools')
library(caTools)
split = sample.split(iris$Species, SplitRatio = 0.75)
training_set = subset(iris, split == TRUE)
test_set = subset(iris, split == FALSE)




In [7]:
# Fitting SVM to the Training set
library(e1071)
classifier = svm(formula = Species ~ ., data = training_set,
                 type = 'C-classification', kernel = 'radial')



In [8]:
summary(classifier)


Call:
svm(formula = Species ~ ., data = training_set, type = "C-classification", 
    kernel = "radial")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.25 

Number of Support Vectors:  42

 ( 7 18 17 )


Number of Classes:  3 

Levels: 
 setosa versicolor virginica




In [9]:
# Predicting the Test set results
y_pred = predict(classifier, newdata = test_set[-5])

# Making the Confusion Matrix
cm = table(test_set[, 5], y_pred)
cm

            y_pred
             setosa versicolor virginica
  setosa         12          0         0
  versicolor      0         11         1
  virginica       0          0        12

In [10]:
# Tune for combos of gamma 0.5,1,2
# and costs 1/10 , 10 , 100
tune.results = tune(svm, train.x = training_set[1:4], train.y =training_set[,5],kernel='radial',
                    ranges=list(cost=10^(-1:2), gamma=c(.5,1,2)))

In [11]:
summary(tune.results)



Parameter tuning of ‘svm’:

- sampling method: 10-fold cross validation 

- best parameters:
 cost gamma
    1   0.5

- best performance: 0.03560606 

- Detailed performance results:
    cost gamma      error dispersion
1    0.1   0.5 0.09621212 0.09754472
2    1.0   0.5 0.03560606 0.06288335
3   10.0   0.5 0.06212121 0.06069018
4  100.0   0.5 0.08030303 0.06692634
5    0.1   1.0 0.09545455 0.06340345
6    1.0   1.0 0.05303030 0.06267512
7   10.0   1.0 0.06212121 0.06069018
8  100.0   1.0 0.07121212 0.05705059
9    0.1   2.0 0.18257576 0.13699117
10   1.0   2.0 0.07121212 0.07135347
11  10.0   2.0 0.07121212 0.05705059
12 100.0   2.0 0.07121212 0.05705059


In [12]:
tuned.classifier = svm(formula = Species ~ ., data = training_set,
                 type = 'C-classification', kernel = 'radial', cost=1, gamma=0.5)



In [13]:
summary(tuned.classifier)




Call:
svm(formula = Species ~ ., data = training_set, type = "C-classification", 
    kernel = "radial", cost = 1, gamma = 0.5)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.5 

Number of Support Vectors:  46

 ( 12 15 19 )


Number of Classes:  3 

Levels: 
 setosa versicolor virginica




In [14]:
# Predicting the Test set results
tuned_y_pred = predict(classifier, newdata = test_set[-5])

# Making the Confusion Matrix
tuned_cm = table(test_set[, 5], tuned_y_pred)
tuned_cm

            tuned_y_pred
             setosa versicolor virginica
  setosa         12          0         0
  versicolor      0         11         1
  virginica       0          0        12